In [0]:
#!pip install category_encoders

In [0]:
import pandas as pd
import numpy as np
import gc
from tqdm import tqdm
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [3]:
%%time
data = pd.read_excel('/content/drive/My Drive/Data/Cleaned_transactions_data.xlsx', converters={'Cardnum':str})

CPU times: user 8.2 s, sys: 114 ms, total: 8.32 s
Wall time: 8.34 s


In [4]:
data.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,0
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803,P,31.42,0
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706,P,178.49,0
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118,P,3.62,0
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,0


In [5]:
%%time
for keys in [['Cardnum', 'Merchnum'], ['Cardnum', 'Merch zip'], ['Cardnum', 'Merch state']]:
    data['_'.join(keys)] = data[keys[0]].apply(str) + '_' + data[keys[1]].apply(str)

CPU times: user 143 ms, sys: 12 ms, total: 155 ms
Wall time: 156 ms


In [6]:
%%time
attributes = ['Cardnum', 'Merchnum', 'Cardnum_Merchnum', 'Cardnum_Merch zip', 'Cardnum_Merch state']
days = ['1d', '2d', '4d', '8d', '15d', '31d']
fns = ['mean', 'max', 'median','sum']
for attribute in attributes:
    attribute_groupby = data.set_index('Date').groupby(attribute)
    for day in tqdm(days):
        attribute_groupby_amt_day = attribute_groupby.rolling(day)['Amount']
        for fn in fns:
            data['Amount_'+attribute+'_'+fn+'_'+day] = data[['Date', attribute]].merge(attribute_groupby_amt_day.agg(fn).reset_index().drop_duplicates(subset = ['Date', attribute]), on = ['Date', attribute], how = 'left')['Amount']
            data['Actual_Amount/'+attribute+'_'+fn+'_'+day] = data['Amount'] / data['Amount_'+attribute+'_'+fn+'_'+day]
        data[attribute+'_count_'+day] = data[['Date', attribute]].merge(attribute_groupby_amt_day.count().reset_index().drop_duplicates(subset = ['Date', attribute]), on = ['Date', attribute], how = 'left')['Amount']
    data['Days_since_'+attribute] = data.groupby(attribute)['Date'].diff() / np.timedelta64(1, 'D')

100%|██████████| 6/6 [05:30<00:00, 55.05s/it]


CPU times: user 35min 49s, sys: 4.63 s, total: 35min 54s
Wall time: 35min 57s


In [7]:
%%time
for attribute in attributes[:2]:
    for day1 in tqdm(days[:2]):
        for day2 in days[-3:]:
            for fn in ['mean', 'count']:
                numerator = data[[attribute, 'Date']].merge(data.set_index('Date').groupby(attribute).rolling(day1)['Amount'].agg(fn).reset_index().drop_duplicates(subset = [attribute, 'Date']), on = [attribute, 'Date'])['Amount']
                denominator = data[[attribute, 'Date']].merge(data.set_index('Date').groupby(attribute).rolling(day2)['Amount'].agg(fn).reset_index(level = 0).groupby(attribute).rolling('1d')['Amount'].mean().reset_index().drop_duplicates(subset = [attribute, 'Date']), on = [attribute, 'Date'])['Amount']
                data[fn+'_'+attribute+'_'+day1+'/'+'mean_'+fn+'_'+attribute+'_'+day2] = numerator / denominator

100%|██████████| 2/2 [05:35<00:00, 167.54s/it]

CPU times: user 6min 18s, sys: 591 ms, total: 6min 18s
Wall time: 6min 19s


In [8]:
data.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud,Cardnum_Merchnum,Cardnum_Merch zip,Cardnum_Merch state,Amount_Cardnum_mean_1d,Actual_Amount/Cardnum_mean_1d,Amount_Cardnum_max_1d,Actual_Amount/Cardnum_max_1d,Amount_Cardnum_median_1d,Actual_Amount/Cardnum_median_1d,Amount_Cardnum_sum_1d,Actual_Amount/Cardnum_sum_1d,Cardnum_count_1d,Amount_Cardnum_mean_2d,Actual_Amount/Cardnum_mean_2d,Amount_Cardnum_max_2d,Actual_Amount/Cardnum_max_2d,Amount_Cardnum_median_2d,Actual_Amount/Cardnum_median_2d,Amount_Cardnum_sum_2d,Actual_Amount/Cardnum_sum_2d,Cardnum_count_2d,Amount_Cardnum_mean_4d,Actual_Amount/Cardnum_mean_4d,Amount_Cardnum_max_4d,Actual_Amount/Cardnum_max_4d,Amount_Cardnum_median_4d,Actual_Amount/Cardnum_median_4d,Amount_Cardnum_sum_4d,Actual_Amount/Cardnum_sum_4d,Cardnum_count_4d,Amount_Cardnum_mean_8d,Actual_Amount/Cardnum_mean_8d,Amount_Cardnum_max_8d,Actual_Amount/Cardnum_max_8d,Amount_Cardnum_median_8d,Actual_Amount/Cardnum_median_8d,Amount_Cardnum_sum_8d,Actual_Amount/Cardnum_sum_8d,Cardnum_count_8d,Amount_Cardnum_mean_15d,Actual_Amount/Cardnum_mean_15d,Amount_Cardnum_max_15d,Actual_Amount/Cardnum_max_15d,Amount_Cardnum_median_15d,Actual_Amount/Cardnum_median_15d,Amount_Cardnum_sum_15d,Actual_Amount/Cardnum_sum_15d,Cardnum_count_15d,Amount_Cardnum_mean_31d,Actual_Amount/Cardnum_mean_31d,Amount_Cardnum_max_31d,Actual_Amount/Cardnum_max_31d,Amount_Cardnum_median_31d,Actual_Amount/Cardnum_median_31d,Amount_Cardnum_sum_31d,Actual_Amount/Cardnum_sum_31d,Cardnum_count_31d,Days_since_Cardnum,Amount_Merchnum_mean_1d,Actual_Amount/Merchnum_mean_1d,Amount_Merchnum_max_1d,Actual_Amount/Merchnum_max_1d,Amount_Merchnum_median_1d,Actual_Amount/Merchnum_median_1d,Amount_Merchnum_sum_1d,Actual_Amount/Merchnum_sum_1d,Merchnum_count_1d,Amount_Merchnum_mean_2d,Actual_Amount/Merchnum_mean_2d,Amount_Merchnum_max_2d,Actual_Amount/Merchnum_max_2d,Amount_Merchnum_median_2d,Actual_Amount/Merchnum_median_2d,Amount_Merchnum_sum_2d,Actual_Amount/Merchnum_sum_2d,Merchnum_count_2d,Amount_Merchnum_mean_4d,Actual_Amount/Merchnum_mean_4d,Amount_Merchnum_max_4d,Actual_Amount/Merchnum_max_4d,Amount_Merchnum_median_4d,Actual_Amount/Merchnum_median_4d,Amount_Merchnum_sum_4d,Actual_Amount/Merchnum_sum_4d,Merchnum_count_4d,Amount_Merchnum_mean_8d,Actual_Amount/Merchnum_mean_8d,Amount_Merchnum_max_8d,Actual_Amount/Merchnum_max_8d,Amount_Merchnum_median_8d,Actual_Amount/Merchnum_median_8d,Amount_Merchnum_sum_8d,Actual_Amount/Merchnum_sum_8d,Merchnum_count_8d,Amount_Merchnum_mean_15d,Actual_Amount/Merchnum_mean_15d,Amount_Merchnum_max_15d,Actual_Amount/Merchnum_max_15d,Amount_Merchnum_median_15d,Actual_Amount/Merchnum_median_15d,Amount_Merchnum_sum_15d,Actual_Amount/Merchnum_sum_15d,Merchnum_count_15d,Amount_Merchnum_mean_31d,Actual_Amount/Merchnum_mean_31d,Amount_Merchnum_max_31d,Actual_Amount/Merchnum_max_31d,Amount_Merchnum_median_31d,Actual_Amount/Merchnum_median_31d,Amount_Merchnum_sum_31d,Actual_Amount/Merchnum_sum_31d,Merchnum_count_31d,Days_since_Merchnum,Amount_Cardnum_Merchnum_mean_1d,Actual_Amount/Cardnum_Merchnum_mean_1d,Amount_Cardnum_Merchnum_max_1d,Actual_Amount/Cardnum_Merchnum_max_1d,Amount_Cardnum_Merchnum_median_1d,Actual_Amount/Cardnum_Merchnum_median_1d,Amount_Cardnum_Merchnum_sum_1d,Actual_Amount/Cardnum_Merchnum_sum_1d,Cardnum_Merchnum_count_1d,Amount_Cardnum_Merchnum_mean_2d,Actual_Amount/Cardnum_Merchnum_mean_2d,Amount_Cardnum_Merchnum_max_2d,Actual_Amount/Cardnum_Merchnum_max_2d,Amount_Cardnum_Merchnum_median_2d,Actual_Amount/Cardnum_Merchnum_median_2d,Amount_Cardnum_Merchnum_sum_2d,Actual_Amount/Cardnum_Merchnum_sum_2d,Cardnum_Merchnum_count_2d,Amount_Cardnum_Merchnum_mean_4d,Actual_Amount/Cardnum_Merchnum_mean_4d,Amount_Cardnum_Merchnum_max_4d,Actual_Amount/Cardnum_Merchnum_max_4d,Amount_Cardnum_Merchnum_median_4d,Actual_Amount/Cardnum_Merchnum_median_4d,Amount_Cardnum_Merchnum_sum_4d,Actual_Amount/Cardnum_Merchnum_sum_4d,Cardnum_Merchnum_count_4d,Amount_Cardnum_Merchnum_mean_8d,Actual_Amount

In [9]:
data.shape

(96397, 312)

In [0]:
data.to_csv('features.csv', index = False)

In [0]:
data.to_csv('/content/drive/My Drive/Data/features.csv', index = False)